1. Install & Load spaCy

In [18]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


2. Import Libraries and Load Model

In [19]:
import spacy

nlp = spacy.load("en_core_web_sm")


3. Real-World Dataset (News + Social Media Style)

In [20]:
data = [
    ("Elon Musk visited India in 2024",
     [("Elon Musk", "PERSON"), ("India", "GPE"), ("2024", "DATE")]),

    ("Apple launched the iPhone in California",
     [("Apple", "ORG"), ("California", "GPE")]),

    ("Narendra Modi met Joe Biden in Washington",
     [("Narendra Modi", "PERSON"), ("Joe Biden", "PERSON"), ("Washington", "GPE")]),

    ("Google announced new AI tools in London",
     [("Google", "ORG"), ("London", "GPE")]),

    ("Cristiano Ronaldo plays for Al Nassr",
     [("Cristiano Ronaldo", "PERSON"), ("Al Nassr", "ORG")])
]


4. Function to Extract Predicted Entities

In [21]:
def get_predictions(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]


5. Compare True vs Predicted Labels

In [22]:
true_labels = []
predicted_labels = []

for text, annotations in data:
    true = annotations
    pred = get_predictions(text)

    # Match entity labels (only labels, not text)
    true_labels.extend([label for _, label in true])
    predicted_labels.extend([label for _, label in pred])


6. Convert Labels for Evaluation

In [23]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

all_labels = true_labels + predicted_labels
encoder.fit(all_labels)

true_encoded = encoder.transform(true_labels)
pred_encoded = encoder.transform(predicted_labels)


7. Evaluation Metrics (Accuracy, Precision, Recall, F1)

In [25]:
def evaluate_ner(data, nlp):
    tp = 0  # true positives
    fp = 0  # false positives
    fn = 0  # false negatives

    for text, true_entities in data:
        doc = nlp(text)
        predicted_entities = [(ent.text, ent.label_) for ent in doc.ents]

        true_set = set(true_entities)
        pred_set = set(predicted_entities)

        tp += len(true_set & pred_set)
        fp += len(pred_set - true_set)
        fn += len(true_set - pred_set)

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    accuracy = tp / (tp + fp + fn) if tp + fp + fn > 0 else 0

    return accuracy, precision, recall, f1


In [26]:
accuracy, precision, recall, f1 = evaluate_ner(data, nlp)

print("Accuracy :", round(accuracy, 3))
print("Precision:", round(precision, 3))
print("Recall   :", round(recall, 3))
print("F1 Score :", round(f1, 3))


Accuracy : 0.625
Precision: 0.714
Recall   : 0.833
F1 Score : 0.769


8. Test on New Real Text

In [27]:
text = "Microsoft CEO Satya Nadella visited Mumbai in January"
doc = nlp(text)

print("Extracted Entities:")
for ent in doc.ents:
    print(ent.text, "->", ent.label_)


Extracted Entities:
Microsoft -> ORG
Satya Nadella -> PERSON
Mumbai -> GPE
January -> DATE
